# Step 1: Exploring & preprocessing raw data

In [1]:
import pandas as pd
df_weather_raw = pd.read_csv('files/weather_data.csv',sep=',',decimal='.')
df_calls_raw = pd.read_csv('files/calls_data.csv',sep=',',decimal='.')

In [2]:
df_weather_raw.head(2)

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1009843200,2002-01-01 00:00:00 +0000 UTC,-28800,Seattle,47.606209,-122.332071,9.21,6.88,6.63,11.0,...,40,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04d
1,1009846800,2002-01-01 01:00:00 +0000 UTC,-28800,Seattle,47.606209,-122.332071,8.20,6.20,4.64,10.0,...,39,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03n


In [3]:
df_weather_raw.describe()

,dt,timezone,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id
count,1.709960e+05,170996.000000,1.709960e+05,1.709960e+05,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,0.0,0.0,170996.000000,170996.000000,170996.000000,33569.000000,4436.000000,836.000000,99.000000,170996.000000,170996.000000
mean,1.307728e+09,-26542.178764,4.760621e+01,-1.223321e+02,11.377896,8.723039,9.295342,12.494648,1017.108985,NaN,NaN,72.768597,2.733629,164.969678,0.825524,2.090532,0.592859,2.026263,59.314241,741.282714
std,1.724745e+08,1740.809420,1.336393e-10,8.824967e-12,6.202178,7.069191,6.360218,6.295122,7.315566,NaN,NaN,16.825505,1.558442,97.741731,1.009982,2.539365,0.474494,1.671953,37.236841,116.000070
min,1.009843e+09,-28800.000000,4.760621e+01,-1.223321e+02,-9.150000,-15.740000,-17.680000,-7.340000,974.000000,NaN,NaN,8.000000,0.010000,0.000000,0.040000,0.300000,0.100000,0.300000,0.000000,200.000000
25%,1.158022e+09,-28800.000000,4.760621e+01,-1.223321e+02,6.920000,3.470000,4.800000,7.967500,1013.000000,NaN,NaN,62.000000,1.540000,110.000000,0.300000,0.500000,0.300000,0.800000,20.000000,800.000000
50%,1.307925e+09,-25200.000000,4.760621e+01,-1.223321e+02,10.890000,8.170000,8.890000,12.000000,1017.000000,NaN,NaN,76.000000,2.570000,160.000000,0.500000,1.000000,0.420000,1.300000,75.000000,801.000000
75%,1.457709e+09,-25200.000000,4.760621e+01,-1.223321e+02,15.450000,13.770000,13.410000,16.700000,1022.000000,NaN,NaN,86.000000,3.600000,215.000000,1.000000,2.800000,0.800000,3.000000,90.000000,803.000000
max,1.604531e+09,-25200.000000,4.760621e+01,-1.223321e+02,38.530000,36.970000,34.880000,40.000000,1045.000000,NaN,NaN,100.000000,31.000000,360.000000,34.800000,24.900000,3.000000,6.600000,100.000000,804.000000


In [4]:
df_calls_raw.head()

,Address,Type,Datetime,Latitude,Longitude,Report Location,Incident Number
0,15TH AV E / E REPUBLICAN ST,Aid Response,11/23/2019 03:41:00 PM,47.623102,-122.312631,POINT (-122.312631 47.623102),F190124488
1,4700 Sw Admiral Way,Aid Response,06/06/2019 04:45:00 AM,47.581214,-122.391903,POINT (-122.391903 47.581214),F190058688
2,1044 Ne 117th St,Trans to AMR,11/23/2019 03:42:00 PM,47.714008,-122.317357,POINT (-122.317357 47.714008),F190124491
3,1711 E Olive Way,Automatic Fire Alarm False,11/23/2019 03:43:00 PM,47.619869,-122.323072,POINT (-122.323072 47.619869),F190124489
4,407 Cedar St,Aid Response,11/23/2019 03:44:00 PM,47.618020,-122.347749,POINT (-122.347749 47.61802),F190124490


Since we are given the hourly information about the weather in Seatle, we won't differentiate the calls for different locations in Seatle. Thus in the call dataset we will remove all the columns, **except the datetime and the incident number** (as we assume there could be a few calls at the same time). <br><br>
When it comes to the data about weather, we will get rid of redundant columns: **dt, timezone, city_name, lat, lon, sea_level, grnd_level, weather_id, weather_icon and weather_main** (as this information is already included in weather_description).  <br>
(A little observation: it's not clear, how there are different timezones in the dataset, while the latitude and longitute do not change.)  

In [5]:
columns_to_drop_weather = ['dt', 'timezone', 'lat', 'lon', 'sea_level', 'grnd_level', 'weather_id', 'weather_icon', \
                   'weather_main', 'city_name']
columns_to_drop_calls = ['Address','Type','Latitude','Longitude','Report Location']
df_weather_raw = df_weather_raw.drop(columns_to_drop_weather,axis = 1) 
df_calls_raw = df_calls_raw.drop(columns_to_drop_calls,axis = 1) 

In [6]:
print('Missing values calls?')
display(df_calls_raw.isnull().any())
print('Missing values weather?')
display(df_weather_raw.isnull().any())

Missing values calls?


Datetime           False
Incident Number    False
dtype: bool

Missing values weather?


dt_iso                 False
temp                   False
feels_like             False
temp_min               False
temp_max               False
pressure               False
humidity               False
wind_speed             False
wind_deg               False
rain_1h                 True
rain_3h                 True
snow_1h                 True
snow_3h                 True
clouds_all             False
weather_description    False
dtype: bool

The missing weather data about the **rain_1h, rain_3h, snow_1h, snow_3h** can be replaced with zeros, which is clear from the values of the weather description. (No rain or snow in the description for the rows with this missing data)

In [7]:
df_weather_raw = df_weather_raw.fillna(0)
display(df_weather_raw.isnull().any())

dt_iso                 False
temp                   False
feels_like             False
temp_min               False
temp_max               False
pressure               False
humidity               False
wind_speed             False
wind_deg               False
rain_1h                False
rain_3h                False
snow_1h                False
snow_3h                False
clouds_all             False
weather_description    False
dtype: bool

In [8]:
df_weather_raw.describe()

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all
count,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000,170996.000000
mean,11.377896,8.723039,9.295342,12.494648,1017.108985,72.768597,2.733629,164.969678,0.162062,0.054233,0.002898,0.001173,59.314241
std,6.202178,7.069191,6.360218,6.295122,7.315566,16.825505,1.558442,97.741731,0.554771,0.526956,0.053004,0.063070,37.236841
min,-9.150000,-15.740000,-17.680000,-7.340000,974.000000,8.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.920000,3.470000,4.800000,7.967500,1013.000000,62.000000,1.540000,110.000000,0.000000,0.000000,0.000000,0.000000,20.000000
50%,10.890000,8.170000,8.890000,12.000000,1017.000000,76.000000,2.570000,160.000000,0.000000,0.000000,0.000000,0.000000,75.000000
75%,15.450000,13.770000,13.410000,16.700000,1022.000000,86.000000,3.600000,215.000000,0.000000,0.000000,0.000000,0.000000,90.000000
max,38.530000,36.970000,34.880000,40.000000,1045.000000,100.000000,31.000000,360.000000,34.800000,24.900000,3.000000,6.600000,100.000000


#### Let's adjust the datestamp of the calls to the standard 'YYYY-MM-DD HH:MM:SS'

In [9]:
df_calls_raw['Datetime'] = pd.to_datetime(df_calls_raw.Datetime)
df_calls_raw.head()

,Datetime,Incident Number
0,2019-11-23 15:41:00,F190124488
1,2019-06-06 04:45:00,F190058688
2,2019-11-23 15:42:00,F190124491
3,2019-11-23 15:43:00,F190124489
4,2019-11-23 15:44:00,F190124490


In [10]:
# for weather data
# convert 2002-01-01 00:00:00 +0000 UTC to 2002-01-01 00:00:00
import re
example= '2002-01-01 00:00:00 +0000 UTC'
pos = example.find('+') # find the position of +
remove_ending = lambda x: x[:pos-1] # remove everything after +
df_weather_raw['dt_iso'] = df_weather_raw['dt_iso'].apply(remove_ending)
df_weather_raw['dt_iso'] = df_weather_raw['dt_iso'].apply(pd.to_datetime)
df_weather_raw.head()

,dt_iso,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_description
0,2002-01-01 00:00:00,9.21,6.88,6.63,11.0,1019,71,1.50,40,0.0,0.0,0.0,0.0,75,broken clouds
1,2002-01-01 01:00:00,8.20,6.20,4.64,10.0,1019,81,1.29,39,0.0,0.0,0.0,0.0,40,scattered clouds
2,2002-01-01 02:00:00,6.61,4.16,4.03,8.5,1019,86,1.73,49,0.0,0.0,0.0,0.0,40,scattered clouds
3,2002-01-01 03:00:00,6.43,3.82,3.29,7.6,1018,87,1.96,65,0.0,0.0,0.0,0.0,40,scattered clouds
4,2002-01-01 04:00:00,6.25,3.36,3.26,7.4,1018,81,2.05,82,0.0,0.0,0.0,0.0,20,few clouds


In [11]:
df_calls_raw.head()

,Datetime,Incident Number
0,2019-11-23 15:41:00,F190124488
1,2019-06-06 04:45:00,F190058688
2,2019-11-23 15:42:00,F190124491
3,2019-11-23 15:43:00,F190124489
4,2019-11-23 15:44:00,F190124490


In [12]:
# remove the dublicates wrt to timestamp for the weather
# since we capture randomly the temperature at one time at one hour, it is fair to choose one value out of two
# but we could also take an average
df_weather_raw = df_weather_raw.drop_duplicates(subset=['dt_iso'])
df_weather_raw =df_weather_raw.reset_index(drop=True)

In [13]:
len(df_weather_raw)

165192

In [14]:
len(df_weather_raw['dt_iso'].unique())

165192

### Let's find out if we are missing any timestamps in the call and weather data

In [15]:
df_calls_raw['tDiff'] = df_calls_raw.Datetime.diff()
df_calls_raw[df_calls_raw.tDiff > pd.Timedelta('1H')]

,Datetime,Incident Number,tDiff
2,2019-11-23 15:42:00,F190124491,170 days 10:57:00
6,2019-10-12 04:26:00,F190108960,114 days 03:48:00
8,2019-09-10 13:38:00,F190097152,21 days 17:46:00
12,2019-07-30 14:40:00,F190080686,89 days 01:38:00
23,2019-06-30 15:37:00,F190068586,10 days 14:35:00
...,...,...,...
1516618,2019-03-20 15:07:09,V190028598,6 days 01:13:28
1516619,2019-03-21 18:22:29,V190029062,1 days 03:15:20
1516620,2019-04-02 14:22:09,V190033312,11 days 19:59:40
1516621,2019-04-06 01:47:26,V190034630,3 days 11:25:17


In [16]:
df_weather_raw['tDiff'] = df_weather_raw.dt_iso.diff()
df_weather_raw[df_weather_raw.tDiff > pd.Timedelta('1H')]

,dt_iso,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_description,tDiff


As we could expect, the fire department doesn't receive calls every hour. This means, that when we combine the calls and weather data, we can simply use **left join** and substitute the missing call values with zeros. 

### We will use the data for the past 5 years: from 2015-11-01 till 2020-11-01

In [17]:
df_calls_raw = df_calls_raw[(df_calls_raw['Datetime']>='2015-11-01 00:00:00') & (df_calls_raw['Datetime']<'2020-11-01 00:00:00')]
df_weather_raw = df_weather_raw[(df_weather_raw['dt_iso']>='2015-11-01 00:00:00') & (df_weather_raw['dt_iso']<'2020-11-01 00:00:00')]

### Next:
Now we will aggregate the information about the calls per hour and join it with the weather information. As suggested, we will be using the database for feature engineering. <br>

Most of the time we could execute the queries from Python, however as it cannot be reproduced, the SQL queries used and the output (csv file) of the queries is going to be provided.<br>
Just for the demonstration, this cell (not excutable) shows an example how we could work with the database from Python, that 

In [18]:
df_weather_raw.head()

,dt_iso,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_description,tDiff
121248,2015-11-01 00:00:00,13.26,10.40,10.57,14.4,1005,80,4.1,170,0.3,0.0,0.0,0.0,90,light rain,01:00:00
121249,2015-11-01 01:00:00,12.93,7.27,10.49,13.9,1005,83,8.2,170,1.0,0.0,0.0,0.0,90,light rain,01:00:00
121250,2015-11-01 02:00:00,12.94,8.88,10.47,13.9,1006,80,5.7,170,0.3,0.0,0.0,0.0,90,light rain,01:00:00
121251,2015-11-01 03:00:00,12.61,8.47,9.80,13.9,1006,80,5.7,180,0.0,1.3,0.0,0.0,75,light rain,01:00:00
121252,2015-11-01 04:00:00,12.23,9.25,9.68,13.3,1006,83,4.1,160,2.8,0.0,0.0,0.0,90,moderate rain,01:00:00


In [19]:
df_calls_raw = df_calls_raw.drop('tDiff',axis = 1)
df_weather_raw = df_weather_raw.drop('tDiff',axis = 1)

In [20]:
# we import these files to use them in the Database
#df_weather_raw.to_csv('weather_import_db.csv') 
#df_calls_raw.to_csv('calls_import_db.csv')

In [21]:
def print_query(file_name):
    print('Would you like to see the query? y/n')
    ans = input()
    if ans=='y':
        with open('files/queries/'+file_name, 'r') as file:
            query_time_series_hour = file.read()
        print('-'*50)
        print(query_time_series_hour)
        print('-'*50)

In [22]:
print_query('time_series_analysis.txt')

Would you like to see the query? y/n
y
--------------------------------------------------
-- create tables
CREATE OR REPLACE TABLE my_schema.weather_data
(
	dt_iso TIMESTAMP,
	temp DECIMAL(16,2),
	feels_like DECIMAL(16,2),
	temp_min DECIMAL(16,2),
	temp_max DECIMAL(16,2),
	pressure DECIMAL(16,0),
	humidity DECIMAL(16,0),
	wind_speed DECIMAL(16,2),
	wind_deg DECIMAL(16,0),
	rain_1h DECIMAL(16,2),
	rain_3h DECIMAL(16,2),
	snow_1h DECIMAL(16,2),
	snow_3h DECIMAL(16,2),
	clouds_all DECIMAL(16,0),
	weather_description VARCHAR(50)
);
CREATE OR REPLACE TABLE my_schema.calls_data
(
	datetime TIMESTAMP,
	incident_number VARCHAR(50)
);

IMPORT INTO my_schema.weather_data
FROM LOCAL CSV FILE 'weather_import_db.csv'
COLUMN SEPARATOR = ';';

IMPORT INTO my_schema.calls_data
FROM LOCAL CSV FILE 'calls_import_db.csv'
COLUMN SEPARATOR = ';';


EXPORT (
WITH calls_info AS
( 
	SELECT 
		TRUNC(datetime,'HH24') date_hour,
		count(incident_number) numb_calls
	FROM my_schema.calls_data
	GROUP BY 1
)
SELECT 